In [105]:
import selenium
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.ui as ui
import selenium.webdriver.support.expected_conditions as EC
import os
import time
from requests import get
from bs4 import BeautifulSoup

In [2]:
import pandas as pd
import numpy as np
Residential = pd.read_csv("Residential.csv")


In [3]:
#Upload the residential.csv to get the property name
Residential.drop(["Unnamed: 0","Unnamed: 7"], axis=1, inplace=True)

In [4]:
Residential.columns

Index(['Median Price Psf', 'Name', 'No of Transactions', 'Street', 'Tenure',
       'Type'],
      dtype='object')

In [5]:
Residential["Type"].value_counts()

CONDOMINIUM              451
FLAT                     238
APARTMENT                163
SERVICE RESIDENCE        143
 CONDOMINIUM               5
 FLAT                      5
 APARTMENT                 4
CONDOMINIUM FLAT           2
 SERVICE RESIDENCE         1
APARTMENT CONDOMINIUM      1
Name: Type, dtype: int64

In [177]:
Test = Residential[1:25]
Test=Test[Test.Type.str.contains("FLAT")==False]

In [ ]:
Test = Residential[Residential.Type.str.contains("FLAT")==False].reset_index()

In [112]:
df_final=[]


In [121]:
Test = Residential.iloc[541:]


In [122]:
from IPython.display import clear_output
driver = webdriver.Chrome('C:/Users/chromedriver.exe') 
driver.get('https://www.propsocial.my');
driver.find_element_by_class_name("dropdown-toggle").click()
username=driver.find_element_by_id("user_email")
password = driver.find_element_by_id("user_password")
username.send_keys("email")
password.send_keys("password")
driver.find_element_by_name("commit").click()

driver.get('https://www.propsocial.my/dashboard')
driver.find_element_by_class_name("propsocial-logo").click()



#get url for current page
url = driver.current_url
response = get(url)


#parse beautifulsoup in the browser
html_soup = BeautifulSoup(response.text, 'html.parser')
#df_final=[]
for name in Test["Name"]:
    search_box = driver.find_element_by_id("search-box")
    search_box.clear()
    search_box.send_keys(name)
    driver.find_element_by_name("button").submit()
    time.sleep(3)
    print(driver.current_url)
    
    url = driver.current_url
    response = get(url)
    findName = html_soup.find("h1").text
    
    if url == "https://www.propsocial.my/malaysia/1/properties":
        name = name
        total_Blocks= "NA"
        total_Storey = "NA"
        total_Units = "NA"
        maintenace = "NA"
        completion_Date = "NA"
        data = {"Name":name,"Block":total_Blocks,"TotalStoreys":total_Storeys,"TotalUnit":total_Units,"Maintenance":
              maintenance,"Complete":completion_Date,"Developer":developer}
        df_final.append(data)
    #parse beautifulsoup in the browser
    else:
        html_soup = BeautifulSoup(response.text, 'html.parser')
        
    #total_Block = html_soup.find("section", class_="property-detail-extras").find("div", class_="value").text

    #find class of total block,total storey,total unit
        blocks =html_soup.find("section", class_="property-detail-extras")
        if blocks:
            total_Blocks = blocks.find_all("div", class_="value")[0].text
        else:
            total_Blocks = "NA"
            
        storeys = html_soup.find("section", class_="property-detail-extras")
        if storeys:
            total_Storeys = storeys.find_all("div", class_="value")[1].text
        else:
            total_Storeys ="NA"
        
        units = html_soup.find("section", class_="property-detail-extras")
        if units:
            total_Units = units.find_all("div", class_="value")[2].text
        else:
            total_Units= "NA"
    #I decided to remove parking as each unit in a particular property may offer different unit of parking
        #parking = html_soup.find("section", class_="property-detail-extras").find_all("div", class_="value")[3].text.split()[1]
        mantain = html_soup.find_all("div", class_="prices-detail")
        if mantain:
            if mantain[3]:
                maintenance = mantain[3].find("span").text.split()[0]
            else:
                maintenance = "NA"
        else:
            maintenance = "NA"
    #Extract date of completion    
        completion = html_soup.find("div", class_="timeframe-div")
        if completion:
            completion_Date=completion.text.split()[0]
        else:
            completion_Date="NA"
            
    #Developer Name
        dev=html_soup.find("div", class_="developer-title")
        if dev:
            developer = dev.text
        else:
            developer = "NA"
        print(name,total_Blocks,total_Storeys,total_Units,maintenance,completion_Date)

        data = {"Name":name,"Block":total_Blocks,"TotalStoreys":total_Storeys,"TotalUnit":total_Units,"Maintenance":
              maintenance,"Complete":completion_Date,"Developer":developer}
        clear_output(wait = True)
        df_final.append(data)

https://www.propsocial.my/malaysia/1/properties
https://www.propsocial.my/malaysia/1/properties


In [81]:
if mantain:
            if mantain[3]:
                maintenance = mantain[3].find("span").text.split()[0]
            else:
                maintenance = "NA"
else:
    maintenance = "NA"
    
maintenance

'NIL'

In [123]:
FINAL = pd.DataFrame(df_final)
FINAL

,Block,Complete,Developer,Maintenance,Name,TotalStoreys,TotalUnit
0,11,2015,Akisama Group,NIL,OUG PARKLANE,NIL,4429
1,NIL,2011,Aston Villa,NIL,THE SCOTT GARDEN,3,NIL
2,4,NA,PPC Glomac Sdn. Bhd.,NIL,SRI PENARA,17,2040
3,NA,NA,PPC Glomac Sdn. Bhd.,NIL,REGALIA SERVICE APARTMENT,17,NA
4,4,2013,Bukit Kiara Properties,RM0.33psf,VERVE SUITES,37,933
5,4,2014,Trinity Group Sdn Bhd,RM0.20psf,THE Z RESIDENCE,27,1136
6,26,2009,Sunrise,RM0.35psf,SOLARIS DUTAMAS,21,2200
7,NIL,1989,NA,NIL,TAMAN MELATI,NIL,NIL
8,NA,NA,NA,NIL,WANGSA MAJU SEKSYEN 2,NIL,NA
9,4,2010,Amatir Resources,RM0.25psf,SENI MONT KIARA,40,605


In [132]:
FINAL = pd.DataFrame(df_final)
#Let check the frequency of the maintenance fee
FINAL.Maintenance.value_counts()

NIL            330
NA             121
RM0.35psf       60
RM0.20psf       49
RM0.25psf       37
RM0.22psf       32
RM0.30psf       31
RM0.40psf       30
RM0.15psf       25
RM0.18psf       19
RM0.33psf       19
RM0.17psf       17
RM0.28psf       16
RM0.16psf       15
RM0.12psf       15
RM0.45psf       11
RM0.37psf       10
RM150/month     10
RM0.55psf       10
RM0.38psf        9
RM0.24psf        9
RM1.10psf        8
RM0.13psf        8
RM0.21psf        7
RM145/month      6
RM76/month       5
RM0.60psf        5
RM0.26psf        4
RM45/month       4
RM0.32psf        4
              ... 
RM100/month      2
RM210/month      2
RM0.42psf        2
RM151/month      2
RM194/month      2
RM221/month      2
RM280/month      2
RM105/month      2
RM0.31psf        1
RM66/month       1
RM380/month      1
RM0.47psf        1
RM250/month      1
RM0.14psf        1
RM80/month       1
RM0.61psf        1
RM175/month      1
RM30/month       1
RM156/month      1
RM0.66psf        1
RM0.23psf        1
RM170/month 

In [124]:
FINAL.to_csv("PropScrap.csv")

In [58]:
df_miss = pd.DataFrame(df_missing)
df_miss.to_csv("df_missing")